In [2]:
import pandas as pd
import random
import os

In [73]:
SEED = 100
OUTPUT_DIR = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data'

In [79]:
random.seed(SEED)

load_dist_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/loads_by_scheduled_time.pkl'
df = pd.read_pickle(load_dist_path)

df['sample_load'] = df['load'].apply(lambda x: random.choice(x))
df = df.reset_index()
df.drop('load', axis=1, inplace=True)
display(df.head())
out_fp = os.path.join(OUTPUT_DIR, 'sampled_load.pkl')
df.to_pickle(out_fp)

,route_id_direction,block_abbr,stop_id_original,time,IsWeekend,sample_load
0,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,0,2.0
1,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,1,3.0
2,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:41,0,7.0
3,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,0,10.0
4,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,1,4.0


In [80]:
random.seed(SEED)

ons_dist_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/ons_by_scheduled_time.pkl'
df = pd.read_pickle(ons_dist_path)

df['sample_ons'] = df['ons'].apply(lambda x: random.choice(x))
df = df.reset_index()
df.drop('ons', axis=1, inplace=True)

display(df.head())
out_fp = os.path.join(OUTPUT_DIR, 'sampled_ons.pkl')
df.to_pickle(out_fp)

,route_id_direction,block_abbr,stop_id_original,time,IsWeekend,sample_ons
0,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,0,0.0
1,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:13,1,0.0
2,14_FROM DOWNTOWN,1400,1SJAMNM,06:18:41,0,0.0
3,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,0,0.0
4,14_FROM DOWNTOWN,1400,1SJAMNM,06:21:13,1,0.0


In [84]:
random.seed(SEED)

tt_dist_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/travel_time_by_scheduled_time.pkl'
df = pd.read_pickle(tt_dist_path)

df['time_to_next_stop'] = df['time_to_next_stop'].apply(lambda x: [item for item in x if item >= 0])
df['has_positive'] = df['time_to_next_stop'].apply(lambda x: len([item for item in x if item >= 0]) > 0)
df = df[df['has_positive'] == True]
df['sample_time_to_next_stop'] = df['time_to_next_stop'].apply(lambda x: random.choice(x))
df = df.reset_index()
df.drop(['time_to_next_stop', 'has_positive'], axis=1, inplace=True)

out_fp = os.path.join(OUTPUT_DIR, 'sampled_travel_times.pkl')
df.to_pickle(out_fp)

In [85]:
import datetime as dt
tdf = df.reset_index().query("route_id_direction == '14_FROM DOWNTOWN' and block_abbr == 1400 and stop_id_original == 'MCC4_20'")
tdf['time'] = pd.to_datetime(tdf['time'], format='%H:%M:%S')
tdf[tdf['time'] == '1900-01-01 14:15:00	']

,index,route_id_direction,block_abbr,stop_sequence,stop_id_original,time,IsWeekend,sample_time_to_next_stop
18,18,14_FROM DOWNTOWN,1400,1,MCC4_20,1900-01-01 14:15:00,0,872.0
19,19,14_FROM DOWNTOWN,1400,1,MCC4_20,1900-01-01 14:15:00,1,855.0


In [3]:
distance_path = '/home/jptalusan/gits/mta_simulator_redo/code_root/scenarios/baseline/data/sampled_travel_times.pkl'
df = pd.read_pickle(distance_path)
df

,route_id_direction,block_abbr,stop_sequence,stop_id_original,time,IsWeekend,sample_time_to_next_stop
0,14_FROM DOWNTOWN,1400,1,MCC4_20,06:15:00,0,498.0
1,14_FROM DOWNTOWN,1400,1,MCC4_20,06:15:00,1,870.0
2,14_FROM DOWNTOWN,1400,1,MCC4_20,06:18:00,0,212.0
3,14_FROM DOWNTOWN,1400,1,MCC4_20,06:18:00,1,584.0
4,14_FROM DOWNTOWN,1400,1,MCC4_20,07:15:00,0,936.0
...,...,...,...,...,...,...,...
533811,9_TO DOWNTOWN,8600,6,2AVJUNSF,07:48:54,0,28.0
533812,9_TO DOWNTOWN,8600,6,2AVJUNSF,07:48:57,0,34.0
533813,9_TO DOWNTOWN,8600,7,2ASTOSM,07:50:02,0,266.0
533814,9_TO DOWNTOWN,8600,7,2ASTOSM,07:50:06,0,328.0


In [8]:
tdf = df.reset_index().query("route_id_direction == '86_FROM NASHVILLE' and block_abbr == 8602 and stop_id_original == 'MCC4_23'")
tdf

,index,route_id_direction,block_abbr,stop_sequence,stop_id_original,time,IsWeekend,sample_time_to_next_stop
513521,513521,86_FROM NASHVILLE,8602,1,MCC4_23,15:48:00,0,412.0


In [13]:
import datetime as dt
a = dt.datetime.strptime("2021-08-23 16:49:00", "%Y-%m-%d %H:%M:%S")
b = dt.datetime.strptime("2021-08-23 17:49:00", "%Y-%m-%d %H:%M:%S")
c = dt.datetime.strptime("2021-08-23 12:00:00", "%Y-%m-%d %H:%M:%S")
d = c + (b-a)
d

datetime.datetime(2021, 8, 23, 13, 0)

In [31]:
np.random.randint(0, 10)

9